In [3]:
import wallaroo
from wallaroo.object import EntityNotFoundError
import pandas as pd
import os
#Used for the Wallaroo SDK version 2023.1
os.environ["ARROW_ENABLED"]="True"

# bigquery library
from google.cloud import bigquery
from google.oauth2 import service_account
import db_dtypes

# wl = wallaroo.Client()

wallarooPrefix = "product-uat-ee"
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

arguments = {}

if "workspace_name" in arguments:
    workspace_name = arguments['workspace_name']
else:
    workspace_name="bigqueryworkspace"

if "pipeline_name" in arguments:
    pipeline_name = arguments['pipeline_name']
else:
    pipeline_name="bigquerypipeline"

if "bigquery_connection_input_name" in arguments:
    bigquery_connection_input_name = arguments['bigquery_connection_input_name']
else:
    bigquery_connection_input_name = "bigqueryhouseinputs"

if "bigquery_connection_output_name" in arguments:
    bigquery_connection_output_name = arguments['bigquery_connection_output_name']
else:
    bigquery_connection_output_name = "bigqueryhouseoutputs"

# helper methods to retrieve workspaces and pipelines

def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

# set the workspace and pipeline
workspace = get_workspace(workspace_name)
wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

# get the connections
big_query_input_connection = wl.get_connection(name=bigquery_connection_input_name)
big_query_output_connection = wl.get_connection(name=bigquery_connection_output_name)

# Set the bigquery input and output credentials
bigquery_input_credentials = service_account.Credentials.from_service_account_info(
    big_query_input_connection.details())

bigquery_output_credentials = service_account.Credentials.from_service_account_info(
    big_query_output_connection.details())

# start the input and output clients
bigqueryinputclient = bigquery.Client(
    credentials=bigquery_input_credentials, 
    project=big_query_input_connection.details()['project_id']
)
bigqueryoutputclient = bigquery.Client(
    credentials=bigquery_output_credentials, 
    project=big_query_output_connection.details()['project_id']
)

# query for the inference input data
inference_dataframe_input = bigqueryinputclient.query(
        f"""
        SELECT tensor
        FROM {big_query_input_connection.details()['dataset']}.{big_query_input_connection.details()['table']}"""
    ).to_dataframe()

# perform the inference and display the first 5 rows
result = pipeline.infer(inference_dataframe_input)
display(result.head(5))

# Get the output table, then upload the inference results
output_table = bigqueryoutputclient.get_table(f"{big_query_output_connection.details()['dataset']}.{big_query_output_connection.details()['table']}")

bigqueryoutputclient.insert_rows_from_dataframe(
    output_table, 
    dataframe=result.rename(columns={"in.tensor":"in_tensor", "out.variable":"out_variable"})
)

# close the bigquery clients
bigqueryinputclient.close()
bigqueryoutputclient.close()


,time,in.tensor,out.variable,check_failures
0,2023-05-10 21:07:08.607,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0,...",[718013.75],0
1,2023-05-10 21:07:08.607,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0,...",[615094.56],0
2,2023-05-10 21:07:08.607,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, ...",[448627.72],0
3,2023-05-10 21:07:08.607,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0,...",[758714.2],0
4,2023-05-10 21:07:08.607,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4....",[513264.7],0
